In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(2)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 1500
piter = 100
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', state_dim_SCON + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_CON)
print(obs_error_CON)

tensor([[4.5660e+01, 4.5647e+01, 4.5650e+01, 4.5643e+01, 4.5638e+01, 4.5640e+01,
         4.5626e+01, 4.5630e+01, 4.5618e+01, 4.5617e+01, 4.5615e+01, 4.5605e+01,
         4.5609e+01, 4.5595e+01, 4.5598e+01, 4.5591e+01],
        [7.1469e-02, 7.3700e-02, 7.3289e-02, 7.4423e-02, 7.5230e-02, 7.4875e-02,
         7.6935e-02, 7.6316e-02, 7.8034e-02, 7.7924e-02, 7.8087e-02, 7.9339e-02,
         7.8534e-02, 8.0440e-02, 7.9792e-02, 8.0558e-02],
        [7.1469e-01, 7.1372e-01, 7.1409e-01, 7.1363e-01, 7.1332e-01, 7.1362e-01,
         7.1266e-01, 7.1317e-01, 7.1240e-01, 7.1253e-01, 7.1251e-01, 7.1189e-01,
         7.1246e-01, 7.1149e-01, 7.1193e-01, 7.1158e-01],
        [1.1000e-03, 1.8381e-03, 4.6086e-04, 3.0443e-03, 4.7108e-04, 1.8966e-03,
         1.1670e-03, 7.0596e-04, 2.7906e-03, 4.2941e-04, 2.8298e-03, 7.3398e-04,
         1.2239e-03, 2.0338e-03, 5.1562e-04, 3.3495e-03]])
tensor([[4.5624e+00, 7.6809e-03, 7.1285e-02, 1.5369e-04]])


In [4]:
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB)
print(obs_error_AWB)

tensor([[5.3602e+01, 5.3588e+01, 5.3591e+01, 5.3584e+01, 5.3578e+01, 5.3579e+01,
         5.3564e+01, 5.3569e+01, 5.3560e+01, 5.3559e+01, 5.3558e+01, 5.3545e+01,
         5.3551e+01, 5.3536e+01, 5.3538e+01, 5.3531e+01],
        [1.1421e-01, 1.1832e-01, 1.1810e-01, 1.1999e-01, 1.2125e-01, 1.2086e-01,
         1.2401e-01, 1.2294e-01, 1.2498e-01, 1.2491e-01, 1.2482e-01, 1.2692e-01,
         1.2544e-01, 1.2826e-01, 1.2713e-01, 1.2800e-01],
        [6.7073e-01, 6.7239e-01, 6.7187e-01, 6.7274e-01, 6.7348e-01, 6.7354e-01,
         6.7549e-01, 6.7470e-01, 6.7601e-01, 6.7623e-01, 6.7639e-01, 6.7818e-01,
         6.7750e-01, 6.7964e-01, 6.7946e-01, 6.8049e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3419e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6757e-03, 5.7473e-04, 2.4797e-03, 5.9680e-04, 1.7478e-03,
         1.2328e-03, 8.2910e-04, 2.4061

In [5]:
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB_ECA)
print(obs_error_AWB_ECA)

tensor([[5.3606e+01, 5.3592e+01, 5.3594e+01, 5.3586e+01, 5.3579e+01, 5.3581e+01,
         5.3567e+01, 5.3572e+01, 5.3559e+01, 5.3558e+01, 5.3555e+01, 5.3545e+01,
         5.3550e+01, 5.3533e+01, 5.3538e+01, 5.3528e+01],
        [1.9081e-01, 1.9506e-01, 1.9527e-01, 1.9779e-01, 1.9962e-01, 1.9968e-01,
         2.0286e-01, 2.0196e-01, 2.0493e-01, 2.0501e-01, 2.0569e-01, 2.0746e-01,
         2.0642e-01, 2.0993e-01, 2.0873e-01, 2.1039e-01],
        [6.7073e-01, 6.7236e-01, 6.7196e-01, 6.7284e-01, 6.7358e-01, 6.7335e-01,
         6.7505e-01, 6.7448e-01, 6.7616e-01, 6.7630e-01, 6.7668e-01, 6.7806e-01,
         6.7749e-01, 6.7978e-01, 6.7921e-01, 6.8067e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3418e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6543e-03, 5.6910e-04, 2.4475e-03, 5.8824e-04, 1.7276e-03,
         1.2063e-03, 8.1481e-04, 2.3599

In [6]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

In [7]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [8]:
#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [9]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [10]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [11]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFLOW(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                #l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                #l1_norm.backward()
                l2_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(x_with_CO2) #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [14]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.370777. Best norm loss value is: 1733.707763671875.

x with CO2 means across time = tensor([[0.9098, 0.7886, 0.7114, 0.0033],
        [0.9469, 0.7867, 0.7200, 0.0033]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.2027e+00, 4.0749e-01, 7.1820e-01, 1.5413e-03],
         [1.5049e+00, 6.3219e-01, 9.2370e-01, 2.4528e-03],
         ...,
         [5.6232e-01, 6.2345e-01, 7.7039e-01, 4.8909e-03],
         [4.0480e-01, 7.2987e-01, 7.3735e-01, 5.6796e-03],
         [7.9279e-01, 6.7355e-01, 4.2597e-01, 5.1943e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [6.9346e-01, 1.0459e+00, 7.6550e-01, 3.7634e-03],
         [1.2337e+00, 6.8979e-01, 5.7171e-01, 2.6054e-03],
         ...,
         [4.5068e-01, 6.7866e-01, 5.5393e-01, 5.2305e-03],
         [7.1351e-01, 5.9124e-01, 6.6172e-01, 4.6415e-03],
         [4.6638e-01, 8.7369e-01, 9.9723e-01, 6.8391e-03]


Train Diffusion:   1%|          | 10/1500 [00:07<17:31,  1.42it/s]

Moving average norm loss at 10 iterations is: 1635.7706909179688. Best norm loss value is: 1579.658935546875.

x with CO2 means across time = tensor([[5.1504e+00, 3.7140e-01, 2.4890e-01, 1.6541e-03],
        [5.1719e+00, 3.7630e-01, 2.5383e-01, 1.6676e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0368e-01, 3.6543e+00, 2.8013e-01, 1.2785e-02],
         [1.2385e+00, 9.7114e-01, 2.2426e-01, 3.5746e-03],
         ...,
         [6.4567e+00, 2.5352e-01, 1.7320e-01, 2.2916e-03],
         [6.9108e+00, 3.0741e-01, 2.0564e-01, 2.7353e-03],
         [8.6880e-01, 3.8432e-01, 3.3636e-01, 3.0148e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.5201e-01, 5.3751e-01, 2.8581e-01, 1.9279e-03],
         [1.7399e+00, 2.3034e+00, 4.1570e-01, 8.4389e-03],
         ...,
         [3.8654e-01, 3.3808e-01, 2.5162e-01, 2.6074e-03],
         [3.8727e-01, 2.3934e-01, 1.7471e-01, 1.8567e-03],
         [3.5820


Train Diffusion:   1%|          | 11/1500 [00:08<18:17,  1.36it/s]


KeyboardInterrupt: 

In [15]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.299671. Best norm loss value is: 1732.9967041015625.

x with CO2 means across time = tensor([[0.9391, 0.7006, 0.7507, 0.0030],
        [0.9525, 0.7107, 0.7367, 0.0030]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.5776e-01, 2.0133e+00, 9.0102e-01, 7.1507e-03],
         [4.6677e-01, 1.3790e+00, 1.6358e+00, 5.2422e-03],
         ...,
         [1.1055e+00, 6.0827e-01, 3.4601e-01, 4.6821e-03],
         [9.5597e-01, 8.3043e-01, 4.9414e-01, 6.3869e-03],
         [4.3676e-01, 1.1315e+00, 3.7265e-01, 8.5737e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.5775e-01, 6.3316e-01, 5.6144e-01, 2.2943e-03],
         [3.3118e-01, 3.3134e-01, 4.2856e-01, 1.2689e-03],
         ...,
         [4.8594e-01, 8.6324e-01, 1.3034e+00, 6.8261e-03],
         [5.1490e-01, 5.2442e-01, 1.7605e+00, 4.4601e-03],
         [1.5677e+00, 3.3821e-01, 3.1938e-01, 2.7048e-03


Train Diffusion:   1%|          | 10/1500 [00:06<15:16,  1.63it/s]

Moving average norm loss at 10 iterations is: 1639.76103515625. Best norm loss value is: 1577.1798095703125.

x with CO2 means across time = tensor([[5.4735e+00, 4.3105e-01, 3.0774e-01, 1.9060e-03],
        [5.1246e+00, 4.3427e-01, 3.0575e-01, 1.9137e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0931e-01, 3.8924e+00, 3.4378e-01, 1.3623e-02],
         [2.8775e-01, 1.9237e-01, 3.9662e-01, 7.5992e-04],
         ...,
         [3.3445e-02, 3.5167e-01, 2.4049e-01, 2.6855e-03],
         [5.0118e+00, 2.7038e-01, 2.8332e-01, 2.3771e-03],
         [9.6187e+00, 2.8555e-01, 4.2210e-01, 2.7906e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.8565e-01, 6.9543e-01, 3.7386e-01, 2.4910e-03],
         [3.2762e+00, 4.2217e+00, 3.2871e-01, 1.5406e-02],
         ...,
         [1.1307e+01, 3.7652e-01, 2.4785e-01, 3.4957e-03],
         [1.5622e-01, 4.0520e-01, 3.0778e-01, 3.1191e-03],
         [6.4195e


Train Diffusion:   1%|          | 11/1500 [00:07<17:08,  1.45it/s]


KeyboardInterrupt: 

In [16]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.286682. Best norm loss value is: 2042.8668212890625.

x with CO2 means across time = tensor([[0.8992, 0.7767, 0.7941, 0.7632, 0.0045],
        [0.9008, 0.7805, 0.7857, 0.7715, 0.0044]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [7.5417e-01, 6.0832e-01, 9.6767e-01, 5.4057e-01, 3.8079e-03],
         [7.9180e-01, 8.1129e-01, 6.2575e-01, 7.4392e-01, 3.6143e-03],
         ...,
         [7.2741e-01, 4.8012e-01, 8.6562e-01, 5.5145e-01, 6.4586e-03],
         [6.4957e-01, 6.6045e-01, 6.3230e-01, 5.3568e-01, 6.6600e-03],
         [1.9472e-01, 1.1443e+00, 1.1585e+00, 4.6523e-01, 1.4693e-02]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [8.4386e-01, 6.1671e-01, 6.4100e-01, 8.5750e-01, 2.9177e-03],
         [9.3034e-01, 6.1515e-01, 1.3345e+00, 8.0324e-01, 4.8289e-03],
         ...,
         [7.3769e-01, 8.0853e-01, 6.0629e-01, 7.17


Train Diffusion:   1%|          | 10/1500 [00:09<22:51,  1.09it/s]

Moving average norm loss at 10 iterations is: 1935.9889770507812. Best norm loss value is: 1860.8828125.

x with CO2 means across time = tensor([[6.1717e+00, 3.5199e-01, 3.8408e-01, 3.9490e-01, 1.4588e-03],
        [5.6366e+00, 3.4067e-01, 3.8014e-01, 3.9878e-01, 1.4395e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [8.2633e-01, 5.6135e+00, 3.3585e-01, 3.4301e-01, 4.5207e-03],
         [3.2865e-01, 2.9691e+00, 3.5673e-01, 3.3861e-01, 4.2482e-03],
         ...,
         [1.5167e+00, 2.8577e-01, 4.6714e-01, 3.4910e-01, 2.7761e-03],
         [4.4061e-01, 2.7567e-01, 4.0597e-01, 4.0245e-01, 2.4334e-03],
         [2.1608e-02, 2.9219e-01, 3.7497e-01, 2.8089e-01, 2.4056e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [8.2633e-01, 1.1542e+00, 3.6781e-01, 4.2630e-01, 2.8050e-03],
         [5.0564e+00, 9.8574e-01, 3.4418e-01, 4.2768e-01, 2.5265e-03],
         ...,
       


Train Diffusion:   1%|          | 11/1500 [00:10<24:45,  1.00it/s]


KeyboardInterrupt: 

In [17]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.950024. Best norm loss value is: 2049.500244140625.

x with CO2 means across time = tensor([[0.7333, 0.7338, 0.7399, 0.7849, 0.0041],
        [0.7269, 0.7324, 0.7281, 0.7688, 0.0040]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.1041e-01, 6.5161e-01, 7.2352e-01, 6.1481e-01, 3.3077e-03],
         [1.0790e+00, 6.4731e-01, 4.9001e-01, 1.1241e+00, 2.5749e-03],
         ...,
         [6.9220e-01, 7.6389e-01, 3.5761e-01, 1.1998e+00, 4.6940e-03],
         [7.6742e-01, 1.1704e+00, 3.8669e-01, 1.2244e-01, 6.4718e-03],
         [6.2313e-01, 1.0520e+00, 8.8284e-01, 1.4347e+00, 1.1584e-02]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.0246e+00, 1.2749e+00, 6.9200e-01, 8.6506e-01, 4.9551e-03],
         [1.0796e+00, 1.3540e+00, 9.8897e-01, 5.2314e-01, 6.9499e-03],
         ...,
         [7.5898e-01, 6.0608e-01, 1.0297e+00, 1.247


Train Diffusion:   1%|          | 10/1500 [00:08<20:28,  1.21it/s]

Moving average norm loss at 10 iterations is: 1960.0778564453126. Best norm loss value is: 1887.2127685546875.

x with CO2 means across time = tensor([[5.2532e+00, 5.3465e-01, 3.8216e-01, 2.4982e-01, 1.9914e-03],
        [5.0282e+00, 5.5057e-01, 3.8578e-01, 2.4682e-01, 2.0383e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [6.7405e-01, 5.5029e+00, 6.3036e-01, 1.6289e-01, 8.1033e-03],
         [7.0883e-01, 9.0949e-02, 3.9073e-01, 3.4423e-01, 4.1614e-04],
         ...,
         [8.4855e-03, 6.4039e-01, 5.2090e-01, 1.1681e-01, 5.6263e-03],
         [7.6458e+00, 5.9620e-01, 3.7345e-01, 1.5770e-01, 4.1334e-03],
         [1.0737e+01, 6.7147e-01, 1.6803e-01, 4.4434e-01, 2.2452e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.8590e+00, 1.0937e+00, 2.1088e-01, 4.1031e-01, 1.6677e-03],
         [4.8968e+00, 4.0137e+00, 4.7180e-01, 2.4718e-01, 5.9896e-03],
         ...,
 


Train Diffusion:   1%|          | 11/1500 [00:09<21:20,  1.16it/s]


KeyboardInterrupt: 

In [18]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.878443. Best norm loss value is: 2048.784423828125.

x with CO2 means across time = tensor([[0.7430, 0.8146, 0.7073, 0.5779, 0.0044],
        [0.7435, 0.8146, 0.7127, 0.5781, 0.0044]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.0529e-01, 5.4360e-01, 7.5434e-01, 6.1489e-01, 2.9736e-03],
         [1.2756e+00, 7.5842e-01, 7.4992e-01, 7.5329e-01, 3.9342e-03],
         ...,
         [4.9149e-01, 7.6819e-01, 6.1650e-01, 4.0998e-01, 7.2396e-03],
         [3.9819e-01, 8.2205e-01, 6.5396e-01, 4.9488e-01, 7.9390e-03],
         [7.9372e-01, 7.1108e-01, 6.7713e-01, 6.9604e-01, 7.3800e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.0529e-01, 8.2601e-01, 8.7079e-01, 6.7695e-01, 4.4445e-03],
         [1.1137e+00, 7.4846e-01, 8.5843e-01, 5.5736e-01, 4.2762e-03],
         ...,
         [3.2659e-01, 8.4987e-01, 6.1290e-01, 5.377


Train Diffusion:   1%|          | 10/1500 [00:08<20:10,  1.23it/s]

Moving average norm loss at 10 iterations is: 1919.6747436523438. Best norm loss value is: 1829.2735595703125.

x with CO2 means across time = tensor([[6.9921e+00, 2.3486e-01, 2.5148e-01, 2.8243e-01, 7.3231e-04],
        [6.3476e+00, 2.2574e-01, 2.5977e-01, 2.7496e-01, 7.5764e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [9.9831e-01, 6.3346e+00, 1.9442e-01, 3.2106e-01, 2.7258e-03],
         [3.7881e+00, 2.6853e+00, 2.0401e-01, 3.0420e-01, 2.4439e-03],
         ...,
         [1.6120e+01, 2.1276e-01, 3.0696e-01, 3.4797e-01, 1.5665e-03],
         [1.2416e+01, 2.0358e-01, 3.1486e-01, 3.2943e-01, 1.5435e-03],
         [1.6899e+01, 1.9291e-01, 3.1728e-01, 2.4429e-01, 1.4836e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [5.8340e-01, 2.7612e+00, 2.7422e-01, 1.9752e-01, 3.1267e-03],
         [9.3919e-01, 8.7212e-01, 2.7774e-01, 2.4419e-01, 1.9549e-03],
         ...,
 


Train Diffusion:   1%|          | 11/1500 [00:10<23:24,  1.06it/s]


KeyboardInterrupt: 

In [19]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.36853. Best norm loss value is: 2043.685302734375.

x with CO2 means across time = tensor([[0.9199, 0.8564, 0.7093, 0.6438, 0.0043],
        [0.8406, 0.8549, 0.6788, 0.6147, 0.0042]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [5.3215e-01, 6.9238e-01, 5.4053e-01, 4.1346e-01, 2.7930e-03],
         [2.1586e+00, 8.1813e-01, 6.7143e-01, 7.2992e-01, 3.8283e-03],
         ...,
         [8.1598e-01, 6.8407e-01, 3.6782e-01, 2.8029e-01, 4.4701e-03],
         [6.4416e-01, 4.9642e-01, 4.0035e-01, 2.4073e-01, 3.8313e-03],
         [5.5017e-01, 7.3204e-01, 4.2677e-01, 7.6898e-02, 5.2973e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [5.3215e-01, 9.3637e-01, 6.2467e-01, 3.6411e-01, 3.8059e-03],
         [9.7890e-01, 1.0234e+00, 7.0826e-01, 4.7593e-01, 4.6040e-03],
         ...,
         [1.2081e+00, 8.0277e-01, 1.0979e+00, 8.7076


Train Diffusion:   1%|          | 10/1500 [00:08<21:22,  1.16it/s]

Moving average norm loss at 10 iterations is: 1929.5915649414062. Best norm loss value is: 1853.614501953125.

x with CO2 means across time = tensor([[6.2624e+00, 3.5367e-01, 2.8173e-01, 2.1587e-01, 1.1643e-03],
        [5.9038e+00, 3.4459e-01, 2.8001e-01, 2.1287e-01, 1.1488e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [2.1293e+00, 4.3017e+00, 3.4768e-01, 2.4774e-01, 4.4115e-03],
         [7.3811e+00, 1.3935e+00, 3.1106e-01, 2.7375e-01, 2.7808e-03],
         ...,
         [1.0004e+01, 3.1167e-01, 2.7564e-01, 1.7464e-01, 1.9729e-03],
         [9.5598e+00, 3.1113e-01, 2.4576e-01, 2.1399e-01, 1.7958e-03],
         [1.0312e+01, 3.3304e-01, 2.2711e-01, 2.0042e-01, 1.7746e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [5.1649e-01, 3.2965e-01, 2.6961e-01, 2.4464e-01, 9.2608e-04],
         [1.2353e+00, 5.4338e-01, 2.9190e-01, 2.1612e-01, 1.4995e-03],
         ...,
  


Train Diffusion:   1%|          | 11/1500 [00:09<21:39,  1.15it/s]


KeyboardInterrupt: 